# Dataset Preparation

Dataset: https://www.kaggle.com/xainano/handwrittenmathsymbols

A function is defined to read an image in single channel (greyscale), invert it to have white object on a black background (for better performance during contour extraction), to extract contours and find their bounding rectangles. The bounding rectangle with the maximum area is selected, resized to 28 by 28, and reshaped to have 784 pixel values.

The function is called on images for numbers 0-9, along with those of addition, subtraction, multiplication and division operators as well as simple parentheses. An additional value is appended to each result returned to label the symbol, with numbers 0-9 marked by their respective digits, and numbers 10 to 15 denoting +, -, /, *, (, and ) respectively. The values of all the images are combined and saved as a dataframe and exported to a CSV file to train the model.

In [1]:
import numpy as np
import pandas as pd
import cv2
import os

In [2]:
def prepare_from(folder):
    train_data=[]
    for filename in os.listdir(folder):
        img=~(cv2.imread(os.path.join(folder,filename),cv2.IMREAD_GRAYSCALE))
        if img is not None:
            ret,thresh=cv2.threshold(img,127,255,cv2.THRESH_BINARY)
            contours=(cv2.findContours(thresh,cv2.RETR_EXTERNAL,cv2.CHAIN_APPROX_NONE))[0]
            sorted_contours=sorted(contours,key=lambda ctr:cv2.boundingRect(ctr)[0])
            h=w=int(28)
            m=0
            for c in sorted_contours:
                x,y,w,h=cv2.boundingRect(c)
                m=max(w*h,m)
                if m==(w*h):
                    x_max=x
                    y_max=y
                    w_max=w
                    h_max=h
            img_crop=thresh[y_max:y_max+h_max,x_max:x_max+w_max]
            img_resize=cv2.resize(img_crop,(28,28))
            img_resize=np.reshape(img_resize,(784,1))
            train_data.append(img_resize)
    return train_data

In [3]:
data=[]

In [4]:
data=prepare_from('data/extracted_images/0')

for i in range(0,len(data)):
    data[i]=np.append(data[i],['0'])
    
print("After Preparing 0 : {} entries".format(len(data)))

After Preparing 0 : 6914 entries


In [5]:
for x in range(1,10):
    data1=prepare_from('data/extracted_images/{}'.format(str(x)))

    for i in range(0,len(data1)):
        data1[i]=np.append(data1[i],['{}'.format(str(x))])

    data=np.concatenate((data,data1))
    print("After Preparing {} : {} entries".format(str(x),len(data)))

After Preparing 1 : 33434 entries
After Preparing 2 : 59575 entries
After Preparing 3 : 70484 entries
After Preparing 4 : 77880 entries
After Preparing 5 : 81425 entries
After Preparing 6 : 84543 entries
After Preparing 7 : 87452 entries
After Preparing 8 : 90520 entries
After Preparing 9 : 94257 entries


In [6]:
symbols={'+':10,'-':11,'forward_slash':12,'times':13,'(':14,')':15}
for x in symbols:
    data1=prepare_from('data/extracted_images/{}'.format(x))

    for i in range(0,len(data1)):
        data1[i]=np.append(data1[i],['{}'.format(str(symbols[x]))])

    data=np.concatenate((data,data1))
    print("After Preparing {} : {} entries".format(x,len(data)))

After Preparing + : 119369 entries
After Preparing - : 153366 entries
After Preparing forward_slash : 153565 entries
After Preparing times : 156816 entries
After Preparing ( : 171110 entries
After Preparing ) : 185465 entries


In [7]:
df=pd.DataFrame(data,index=None)
df.to_csv('train.csv',index=False)

# Model Creation and Training

A Convolutional Neural Network (CNN) model is implemented using the Keras library (based on TensorFlow 2) and trained using the CSV file. The trained model is saved in JSON format along with its weights for future use.

In [8]:
from tensorflow.keras import layers,models,utils,backend
backend.set_image_data_format('channels_last')

In [9]:
model=models.Sequential()
model.add(layers.Conv2D(30,(5,5),input_shape=(28,28,1),activation='relu'))
model.add(layers.MaxPooling2D(pool_size=(2,2)))
model.add(layers.Conv2D(15,(3,3),activation='relu'))
model.add(layers.MaxPooling2D(pool_size=(2,2)))
model.add(layers.Dropout(0.2))
model.add(layers.Flatten())
model.add(layers.Dense(128,activation='relu'))
model.add(layers.Dense(50,activation='relu'))
model.add(layers.Dense(16,activation='softmax'))
model.compile(optimizer='adam',loss='categorical_crossentropy',metrics=['accuracy'])

In [10]:
df_train=pd.read_csv('train.csv',index_col=False)
df_train

,0,1,2,3,4,5,6,7,8,9,...,775,776,777,778,779,780,781,782,783,784
0,0,0,0,0,0,0,0,0,0,164,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,127,255,238,178,178,238,141,...,61,0,0,0,0,0,0,0,0,0
2,0,0,255,178,178,178,178,178,178,178,...,77,77,77,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,22,...,44,0,0,0,0,0,0,0,0,0
4,0,0,0,0,22,178,178,178,178,140,...,178,223,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
185460,178,178,187,207,226,245,223,159,96,32,...,0,0,0,0,0,0,0,0,0,15
185461,178,163,120,77,77,77,55,22,0,0,...,244,182,73,0,0,0,0,0,0,15
185462,178,178,178,178,178,178,165,140,115,90,...,0,0,0,0,0,0,0,0,0,15
185463,0,0,0,0,0,0,22,67,111,155,...,0,0,0,0,0,0,0,0,0,15


In [11]:
labels=df_train[['784']]
cat=utils.to_categorical(labels,num_classes=16)

In [12]:
df_train.drop(columns='784',inplace=True)

In [13]:
l=[]
for i in range(df_train.shape[0]):
    l.append(np.array(df_train[i:i+1]).reshape(28,28,1))

In [14]:
model.fit(np.array(l), cat, epochs=20, batch_size=200,shuffle=True,verbose=1)

Epoch 1/20
928/928 [==============================] - 11s 5ms/step - loss: 1.4246 - accuracy: 0.7392
Epoch 2/20
928/928 [==============================] - 5s 5ms/step - loss: 0.1471 - accuracy: 0.9562
Epoch 3/20
928/928 [==============================] - 5s 5ms/step - loss: 0.1047 - accuracy: 0.9676
Epoch 4/20
928/928 [==============================] - 5s 5ms/step - loss: 0.0858 - accuracy: 0.9742
Epoch 5/20
928/928 [==============================] - 5s 5ms/step - loss: 0.0744 - accuracy: 0.9767
Epoch 6/20
928/928 [==============================] - 5s 5ms/step - loss: 0.0664 - accuracy: 0.9784
Epoch 7/20
928/928 [==============================] - 5s 5ms/step - loss: 0.0610 - accuracy: 0.9806
Epoch 8/20
928/928 [==============================] - 5s 5ms/step - loss: 0.0550 - accuracy: 0.9827
Epoch 9/20
928/928 [==============================] - 5s 5ms/step - loss: 0.0515 - accuracy: 0.9831
Epoch 10/20
928/928 [==============================] - 5s 5ms/step - loss: 0.0491 - accuracy: 0.984

In [15]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 24, 24, 30)        780       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 12, 12, 30)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 10, 10, 15)        4065      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 5, 5, 15)          0         
_________________________________________________________________
dropout (Dropout)            (None, 5, 5, 15)          0         
_________________________________________________________________
flatten (Flatten)            (None, 375)               0         
_________________________________________________________________
dense (Dense)                (None, 128)               4

In [16]:
model_json = model.to_json()
with open("model.json", "w") as json_file:
    json_file.write(model_json)
model.save_weights("model.h5")